In [0]:
dbutils.widgets.text('p_file_date', '2021-03-21')
v_file_date = dbutils.widgets.get('p_file_date')

In [0]:
%run ../includes/configuration

In [0]:
%run ../includes/common_functions 

In [0]:
drivers_df = spark.read.format('delta').load(f'{processed_folder_path}/drivers') \
    .withColumnRenamed('name', 'driver_name') \
    .withColumnRenamed('number', 'driver_number') \
    .withColumnRenamed('nationality', 'driver_nationality') 


constructors_df = spark.read.format('delta').load(f'{processed_folder_path}/constructors') \
    .withColumnRenamed('name', 'team') 


circuits_df = spark.read.format('delta').load(f'{processed_folder_path}/circuits') \
    .withColumnRenamed('location', 'circuit_location')


races_df = spark.read.format('delta').load(f'{processed_folder_path}/races') \
    .withColumnRenamed('name', 'race_name') \
    .withColumnRenamed('race_timestamp', 'race_date') 

results_df = spark.read.format('delta').load(f'{processed_folder_path}/results') \
    .filter(f"file_date = '{v_file_date}'") \
    .withColumnRenamed('time', 'race_time') \
    .withColumnRenamed('race_id', 'result_race_id') \
    .withColumnRenamed('file_date', 'result_file_date')               

##Join circuis to races

In [0]:
circuit_races_df = races_df.join(circuits_df, races_df.circuit_id == circuits_df.circuit_Id, 'inner') \
    .select(races_df.race_id, races_df.race_year, races_df.race_name, races_df.race_date, circuits_df.circuit_location)

### Join results to all other dataframes

In [0]:
race_results_df = results_df.join(circuit_races_df, results_df.result_race_id == circuit_races_df.race_id) \
    .join(drivers_df, results_df.driver_id == drivers_df.driver_id) \
    .join(constructors_df, results_df.constructor_id == constructors_df.constructor_id)

In [0]:
from pyspark.sql.functions import current_timestamp

In [0]:
final_df = race_results_df.select('race_id', 'race_year', 'race_name', 'race_date', 'circuit_location', 'driver_name', 'driver_number', 'driver_nationality', 'team', 'grid', 'fastest_lap', 'race_time', 'points', 'position', 'result_file_date') \
    .withColumn('created_date', current_timestamp()) \
    .withColumnRenamed('result_file_date', 'file_date')


In [0]:
merge_condition = 'tgt.driver_name = src.driver_name AND tgt.race_id = src.race_id'
merge_delta(final_df, 'f1_presentation', 'race_results', presentation_folder_path, merge_condition, 'race_id')

In [0]:
%sql select race_id, count(1) from f1_presentation.race_results group by race_id order by race_id desc;